In [1]:
import glob
import pandas as pd
import json
import requests

In [2]:
df = pd.concat([pd.read_csv(f) for f in glob.glob("./data/*.csv")], ignore_index=True) # as of Jan 2022
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN


In [3]:
df.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        709054
dtype: int64

In [4]:
df = df.dropna() # drop remaining_lease missing values
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

Get MRT Coordinates:

In [5]:
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station',
    'Woodlands North MRT Station',
    'Woodlands South MRT Station',
    'Springleaf MRT Station',
    'Lentor MRT Station',
    'Mayflower MRT Station',
    'Bright Hill MRT Station',
    'Upper Thomson MRT Station'
]

In [6]:
mrt_lat = []
mrt_long = []

for i in range(0, len(list_of_mrt)):
    query_address = list_of_mrt[i]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_mrt=json.loads(resp.content)
    
    if data_mrt['found'] != 0:
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_lat.append('NotFound')
        mrt_lat.append('NotFound')
        print ("No Results")

Jurong East MRT Station,Lat: 1.33315281585758 Long: 103.742286332403
Bukit Batok MRT Station,Lat: 1.34903331201636 Long: 103.749566478309
Bukit Gombak MRT Station,Lat: 1.35861159094192 Long: 103.751790910733
Choa Chu Kang MRT Station,Lat: 1.38536316540225 Long: 103.744370779756
Yew Tee MRT Station,Lat: 1.39753506936297 Long: 103.747405150236
Kranji MRT Station,Lat: 1.42508698073648 Long: 103.762137459497
Marsiling MRT Station,Lat: 1.43252114855026 Long: 103.774074641403
Woodlands MRT Station,Lat: 1.43605761708128 Long: 103.787938777173
Admiralty MRT Station,Lat: 1.44058856161847 Long: 103.800990519771
Sembawang MRT Station,Lat: 1.44905082158502 Long: 103.820046140211
Canberra MRT Station,Lat: 1.44307664075699 Long: 103.829702590959
Yishun MRT Station,Lat: 1.42944308477331 Long: 103.835005047246
Khatib MRT Station,Lat: 1.41738337009565 Long: 103.832979908243
Yio Chu Kang MRT Station,Lat: 1.38175587099132 Long: 103.84494727118
Ang Mo Kio MRT Station,Lat: 1.36993284962264 Long: 103.849558

Woodlands South MRT Station,Lat: 1.42739605281442 Long: 103.79326432221
Springleaf MRT Station,Lat: 1.39758106126824 Long: 103.817856949947
Lentor MRT Station,Lat: 1.38550657972169 Long: 103.835743809669
Mayflower MRT Station,Lat: 1.37146318831015 Long: 103.836567766945
Bright Hill MRT Station,Lat: 1.36328318545185 Long: 103.832893619992
Upper Thomson MRT Station,Lat: 1.35441643365401 Long: 103.832898468504


In [7]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})

In [8]:
df['address'] = df['block'] + " " + df['street_name']
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,address
656851,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70,174 ANG MO KIO AVE 4
656852,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,275000.0,65,541 ANG MO KIO AVE 10
656853,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,285000.0,64,163 ANG MO KIO AVE 4
656854,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,290000.0,63,446 ANG MO KIO AVE 10
656855,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,290000.0,64,557 ANG MO KIO AVE 10
...,...,...,...,...,...,...,...,...,...,...,...,...
812412,2022-01,YISHUN,EXECUTIVE,392,YISHUN AVE 6,01 TO 03,142.0,Apartment,1988,660000.0,65 years 06 months,392 YISHUN AVE 6
812413,2022-01,YISHUN,EXECUTIVE,324,YISHUN CTRL,01 TO 03,142.0,Apartment,1988,750000.0,65 years 07 months,324 YISHUN CTRL
812414,2022-01,YISHUN,EXECUTIVE,325,YISHUN CTRL,10 TO 12,146.0,Maisonette,1988,709000.0,65 years 11 months,325 YISHUN CTRL
812415,2022-01,YISHUN,EXECUTIVE,640A,YISHUN ST 61,01 TO 03,159.0,Apartment,1992,800000.0,69 years 06 months,640A YISHUN ST 61


Get coordinates of each HDB Resale flat in order to conduct the distance from MRT stations as well as CBD:

In [9]:
address_list = df['address'].unique()

In [10]:
latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []
count = 0

for row in range(len(address_list)): 
    query_address = address_list[row]
    query_string='https://developers.onemap.sg/commonapi/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)          

     
    data_geo_location=json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        blk_no.append(data_geo_location['results'][0]['BLK_NO'])
        road_name.append(data_geo_location['results'][0]['ROAD_NAME'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        address.append(query_address)
        print (str(query_address) + " ,Lat: " + data_geo_location['results'][0]['LATITUDE'] + " Long: " + data_geo_location['results'][0]['LONGITUDE'])
    else:
        print ("No Results")

174 ANG MO KIO AVE 4 ,Lat: 1.37509746867904 Long: 103.83761896123
541 ANG MO KIO AVE 10 ,Lat: 1.37392238703482 Long: 103.855621370524
163 ANG MO KIO AVE 4 ,Lat: 1.37354853919927 Long: 103.838176471398
446 ANG MO KIO AVE 10 ,Lat: 1.36776095130953 Long: 103.855357145908
557 ANG MO KIO AVE 10 ,Lat: 1.3716257020332 Long: 103.857736107527
603 ANG MO KIO AVE 5 ,Lat: 1.38020079047279 Long: 103.83575571651
709 ANG MO KIO AVE 8 ,Lat: 1.37113720765377 Long: 103.847662320064
333 ANG MO KIO AVE 1 ,Lat: 1.3613425564061 Long: 103.851698621454
109 ANG MO KIO AVE 4 ,Lat: 1.3700965375834 Long: 103.837687766047
564 ANG MO KIO AVE 3 ,Lat: 1.36984837555524 Long: 103.859404131956
218 ANG MO KIO AVE 1 ,Lat: 1.36511908595886 Long: 103.841742478489
556 ANG MO KIO AVE 10 ,Lat: 1.37203236308683 Long: 103.857625375797
156 ANG MO KIO AVE 4 ,Lat: 1.37549519574964 Long: 103.839947470774
471 ANG MO KIO AVE 10 ,Lat: 1.36346600647245 Long: 103.856702918462
434 ANG MO KIO AVE 10 ,Lat: 1.3678915069194 Long: 103.85345502

405 BEDOK NTH AVE 3 ,Lat: 1.32871338222698 Long: 103.93409540275
172 BEDOK STH RD ,Lat: 1.32066917100909 Long: 103.94468952936
406 BEDOK NTH AVE 3 ,Lat: 1.32822550655778 Long: 103.934469289425
55 NEW UPP CHANGI RD ,Lat: 1.32509255644891 Long: 103.942110168405
66 BEDOK STH AVE 3 ,Lat: 1.31849377884951 Long: 103.943872460454
103 LENGKONG TIGA ,Lat: 1.32585546338587 Long: 103.910598910975
67 BEDOK STH AVE 3 ,Lat: 1.31897410282406 Long: 103.944076222052
114 BEDOK RESERVOIR RD ,Lat: 1.33056065989658 Long: 103.909838947885
10D BEDOK STH AVE 2 ,Lat: 1.3217708591015 Long: 103.936326570445
629 BEDOK RESERVOIR RD ,Lat: 1.33439908281466 Long: 103.914501162624
104 LENGKONG TIGA ,Lat: 1.32563496745023 Long: 103.909815198327
308 SHUNFU RD ,Lat: 1.35058252902296 Long: 103.837846846026
271 BISHAN ST 24 ,Lat: 1.35661576031593 Long: 103.84231764069
107 BISHAN ST 12 ,Lat: 1.34641761420969 Long: 103.847920706844
239 BISHAN ST 22 ,Lat: 1.36043776070807 Long: 103.844783880156
264 BISHAN ST 24 ,Lat: 1.358668

50 HOY FATT RD ,Lat: 1.2878771583612 Long: 103.811440922662
106B DEPOT RD ,Lat: 1.28064950442563 Long: 103.810750913525
95 HAVELOCK RD ,Lat: 1.28847543074122 Long: 103.832848165751
82 REDHILL LANE ,Lat: 1.28742210332915 Long: 103.81943492781
119B KIM TIAN RD ,Lat: 1.28115558487763 Long: 103.827266559741
75C REDHILL RD ,Lat: 1.28902314290104 Long: 103.816276016167
121 KIM TIAN PL ,Lat: 1.28270270194687 Long: 103.828491163221
2C BOON TIONG RD ,Lat: 1.2868576787328 Long: 103.832845909249
109 GANGSA RD ,Lat: 1.37736736489734 Long: 103.767748758874
142 PETIR RD ,Lat: 1.37639172444397 Long: 103.767912823562
251 BANGKIT RD ,Lat: 1.38030045457366 Long: 103.774174995217
409 SAUJANA RD ,Lat: 1.38233492640361 Long: 103.767677663417
208 PETIR RD ,Lat: 1.3739808401432 Long: 103.775028040256
435 BT PANJANG RING RD ,Lat: 1.38603975530039 Long: 103.770196329443
412 SAUJANA RD ,Lat: 1.38289762281394 Long: 103.768275120432
466 SEGAR RD ,Lat: 1.38694295549199 Long: 103.773299109481
140 PETIR RD ,Lat: 1.3

351 CLEMENTI AVE 2 ,Lat: 1.31535006995344 Long: 103.770861377512
429 CLEMENTI AVE 3 ,Lat: 1.31291218870832 Long: 103.762860973019
304 CLEMENTI AVE 4 ,Lat: 1.32093420856687 Long: 103.764986457895
307 CLEMENTI AVE 4 ,Lat: 1.321415896635 Long: 103.76710548935
308 CLEMENTI AVE 4 ,Lat: 1.32105185036523 Long: 103.766910446634
374 CLEMENTI AVE 4 ,Lat: 1.31912618587195 Long: 103.766983653394
434 CLEMENTI AVE 3 ,Lat: 1.31456889716041 Long: 103.761612603199
729 CLEMENTI WEST ST 2 ,Lat: 1.30492724290307 Long: 103.763519401903
331 CLEMENTI AVE 2 ,Lat: 1.31413785421482 Long: 103.767942210615
511 WEST COAST DR ,Lat: 1.31032016677093 Long: 103.760411562158
318 CLEMENTI AVE 4 ,Lat: 1.31803746016205 Long: 103.765413358653
605 CLEMENTI WEST ST 1 ,Lat: 1.30538490099306 Long: 103.768020281
342 CLEMENTI AVE 5 ,Lat: 1.31836321447687 Long: 103.769846484274
43 CIRCUIT RD ,Lat: 1.32648228290473 Long: 103.885544901533
109 ALJUNIED CRES ,Lat: 1.31939291785822 Long: 103.884439876207
82 MACPHERSON LANE ,Lat: 1.332

480 JURONG WEST ST 41 ,Lat: 1.34684766009461 Long: 103.724041804477
473 JURONG WEST ST 41 ,Lat: 1.34883175011115 Long: 103.722734202036
504 JURONG WEST ST 51 ,Lat: 1.34982761313556 Long: 103.718843750842
440 JURONG WEST AVE 1 ,Lat: 1.35131146892729 Long: 103.722836259059
463 JURONG WEST ST 41 ,Lat: 1.35057187534259 Long: 103.723239272388
915 JURONG WEST ST 91 ,Lat: 1.34108918953192 Long: 103.687870332112
916 JURONG WEST ST 91 ,Lat: 1.34069322660253 Long: 103.687815550891
914 JURONG WEST ST 91 ,Lat: 1.34145537092651 Long: 103.687970250455
475 JURONG WEST ST 41 ,Lat: 1.34809804107273 Long: 103.725087226378
462 JURONG WEST ST 41 ,Lat: 1.35065966569769 Long: 103.722348235699
339A KANG CHING RD ,Lat: 1.33785466388305 Long: 103.721462722132
337B TAH CHING RD ,Lat: 1.33745034737284 Long: 103.722395565486
276A JURONG WEST ST 25 ,Lat: 1.35320439038387 Long: 103.703654377512
273D JURONG WEST AVE 3 ,Lat: 1.35225502596369 Long: 103.70308133299
275B JURONG WEST ST 25 ,Lat: 1.35288372552565 Long: 10

...

264A PUNGGOL WAY ,Lat: 1.40597704114999 Long: 103.898544055197
453D FERNVALE RD ,Lat: 1.39341707295426 Long: 103.875340178233
279A SENGKANG EAST AVE ,Lat: 1.38577466101388 Long: 103.893065739187
440A FERNVALE LINK ,Lat: 1.39318853941818 Long: 103.878692030242
329A ANCHORVALE ST ,Lat: 1.39619302195076 Long: 103.88972181068
453C FERNVALE RD ,Lat: 1.39353259342993 Long: 103.875029902679
279B SENGKANG EAST AVE ,Lat: 1.38541559450444 Long: 103.892986078183
519C TAMPINES CTRL 8 ,Lat: 1.35621338867608 Long: 103.939960455271
159 SIMEI RD ,Lat: 1.34480182914055 Long: 103.959861576671
317C YISHUN AVE 9 ,Lat: 1.43378048287839 Long: 103.843105705893
431A YISHUN AVE 1 ,Lat: 1.41960682497896 Long: 103.84728823711
317A YISHUN AVE 9 ,Lat: 1.43450592896688 Long: 103.843292004282
318B YISHUN AVE 9 ,Lat: 1.43367833611169 Long: 103.843871260947
318A YISHUN AVE 9 ,Lat: 1.43407449545002 Long: 103.843573649375
513 BT BATOK ST 52 ,Lat: 1.35405485249506 Long: 103.752320202563
547C SEGAR RD ,Lat: 1.388538272774

494E TAMPINES ST 43 ,Lat: 1.36421531545345 Long: 103.953511202271
863B TAMPINES ST 83 ,Lat: 1.35546395095099 Long: 103.935138528274
126 POTONG PASIR AVE 1 ,Lat: 1.33497246267555 Long: 103.865234309552
180A MARSILING RD ,Lat: 1.43904320861563 Long: 103.778109726656
505A YISHUN ST 51 ,Lat: 1.41884459566005 Long: 103.84482453185
748B BEDOK RESERVOIR CRES ,Lat: 1.33629391354583 Long: 103.921242294579
747C BEDOK RESERVOIR CRES ,Lat: 1.33679206446803 Long: 103.921982613627
46 TELOK BLANGAH DR ,Lat: 1.27169061066277 Long: 103.809852257535
144 PETIR RD ,Lat: 1.37581593705491 Long: 103.767563450683
311B CLEMENTI AVE 4 ,Lat: 1.31944669077178 Long: 103.765649446368
311C CLEMENTI AVE 4 ,Lat: 1.32010579446331 Long: 103.765573473984
311A CLEMENTI AVE 4 ,Lat: 1.31911656776897 Long: 103.765492195959
475A UPP SERANGOON CRES ,Lat: 1.37629840280721 Long: 103.900220786851
477B UPP SERANGOON VIEW ,Lat: 1.37881205219672 Long: 103.904229621366
47 JLN BAHAGIA ,Lat: 1.32750482242864 Long: 103.856897098071
527D

467A FERNVALE LINK ,Lat: 1.39798359035227 Long: 103.8784589917
471A FERNVALE ST ,Lat: 1.39604094734393 Long: 103.880735419984
472B FERNVALE ST ,Lat: 1.39677177049675 Long: 103.881104301377
471B FERNVALE ST ,Lat: 1.39649875629749 Long: 103.880447337309
293D COMPASSVALE CRES ,Lat: 1.39674216564884 Long: 103.895331584899
468C FERNVALE LINK ,Lat: 1.39689621112725 Long: 103.877785328873
871C TAMPINES ST 86 ,Lat: 1.35670900289357 Long: 103.931385349285
808C CHAI CHEE RD ,Lat: 1.32789302968468 Long: 103.923874941007
96A HENDERSON RD ,Lat: 1.28798878836651 Long: 103.821947260359
442B FAJAR RD ,Lat: 1.38143384735951 Long: 103.771366304235
442C FAJAR RD ,Lat: 1.38148756854256 Long: 103.770942229271
18A CIRCUIT RD ,Lat: 1.32932082372457 Long: 103.888071748726
316 HOUGANG AVE 7 ,Lat: 1.36497584742043 Long: 103.896809883055
57 TEBAN GDNS RD ,Lat: 1.32254226544254 Long: 103.737560699176
935 JURONG WEST ST 91 ,Lat: 1.34267202573949 Long: 103.688333030285
256C SUMANG WALK ,Lat: 1.40540203681824 Long: 

162B RIVERVALE CRES ,Lat: 1.39035967745112 Long: 103.908076555691
474 TAMPINES ST 43 ,Lat: 1.35987299160234 Long: 103.952787602577
890A WOODLANDS DR 50 ,Lat: 1.43828221158378 Long: 103.793216471148
333 WOODLANDS ST 32 ,Lat: 1.42976674719237 Long: 103.780001772636
107 LENGKONG TIGA ,Lat: 1.32495459884083 Long: 103.910175149612
147 BT BATOK WEST AVE 6 ,Lat: 1.34957751809962 Long: 103.743601296461
95A HENDERSON RD ,Lat: 1.28649254110004 Long: 103.82143358771
811A CHOA CHU KANG AVE 7 ,Lat: 1.3737308481902 Long: 103.745929784531
803C KEAT HONG CL ,Lat: 1.37338731917488 Long: 103.744026034489
805B KEAT HONG CL ,Lat: 1.37274770355775 Long: 103.74315742408
816A KEAT HONG LINK ,Lat: 1.37603151881569 Long: 103.748824632963
816B KEAT HONG LINK ,Lat: 1.37604113515339 Long: 103.748460043787
505 HOUGANG AVE 8 ,Lat: 1.37305966832955 Long: 103.887655080049
473A UPP SERANGOON CRES ,Lat: 1.37682525094625 Long: 103.900641038235
473C UPP SERANGOON CRES ,Lat: 1.37628006481899 Long: 103.901387559
116A JLN T

364A UPP SERANGOON RD ,Lat: 1.37060840744452 Long: 103.896776019259
365B UPP SERANGOON RD ,Lat: 1.36920419004326 Long: 103.897512942583
365A UPP SERANGOON RD ,Lat: 1.36971560040268 Long: 103.897624425125
113C MCNAIR RD ,Lat: 1.32088235344697 Long: 103.860331710143
410 PASIR RIS DR 6 ,Lat: 1.37281637140974 Long: 103.953933460075
545 PASIR RIS ST 51 ,Lat: 1.36903491024162 Long: 103.95243622672
119B CANBERRA CRES ,Lat: 1.44637013931729 Long: 103.831574876885
288B COMPASSVALE CRES ,Lat: 1.39844660137875 Long: 103.897807553142
287A COMPASSVALE CRES ,Lat: 1.39843751923085 Long: 103.895957792826
288A COMPASSVALE CRES ,Lat: 1.39864922039269 Long: 103.89700196712
119 SERANGOON NTH AVE 1 ,Lat: 1.36748142862257 Long: 103.871491752337
671B YISHUN AVE 4 ,Lat: 1.41878341784758 Long: 103.842760847841
529 ANG MO KIO AVE 10 ,Lat: 1.37234358233883 Long: 103.855455428073
101 LENGKONG TIGA ,Lat: 1.32515983382087 Long: 103.910888819332
208A CLEMENTI AVE 6 ,Lat: 1.32200240528559 Long: 103.762448319374
315 H

In [11]:
df_coordinates = pd.DataFrame({
    'latitude': latitude,
    'longitude': longitude,
    'blk_no': blk_no,
    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)

9218

In [12]:
list_of_lat = df_coordinates['latitude']
list_of_long = df_coordinates['longitude']
mrt_lat = mrt_location['latitude']
mrt_long = mrt_location['longitude']

In [13]:
list_of_coordinates = []
list_of_mrt_coordinates = []

for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long))

Getting the distances via OneMap API:

In [14]:
# Distance to nearest MRT
from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()

In [15]:
# Distance from CDB
cbd_dist = []

for origin in list_of_coordinates:
    cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters) #CBD coordinates

In [16]:
# Put MRT and CBD distance together
df_coordinates['cbd_dist'] = cbd_dist
df_coordinates['min_dist_mrt'] = min_dist_mrt

In [17]:
df_coordinates

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt
0,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691
1,1.37392238703482,103.855621370524,541,ANG MO KIO AVENUE 10,560541,541 ANG MO KIO AVE 10,10065.227417,806.176167
2,1.37354853919927,103.838176471398,163,ANG MO KIO AVENUE 4,560163,163 ANG MO KIO AVE 4,10118.355593,291.928107
3,1.36776095130953,103.855357145908,446,ANG MO KIO AVENUE 10,560446,446 ANG MO KIO AVE 10,9383.300907,688.600961
4,1.3716257020332,103.857736107527,557,ANG MO KIO AVENUE 10,560557,557 ANG MO KIO AVE 10,9825.917255,929.164159
...,...,...,...,...,...,...,...,...
9213,1.37592215469112,103.773000511438,224,PENDING ROAD,670224,224 PENDING RD,13472.386233,1167.786953
9214,1.40268497999974,103.893608971394,226B,SUMANG LANE,822226,226B SUMANG LANE,14046.815910,1233.055078
9215,1.33541493957748,103.865168758339,125,POTONG PASIR AVENUE 1,350125,125 POTONG PASIR AVE 1,5997.772735,645.899682
9216,1.41859080728129,103.841823925238,672B,YISHUN AVENUE 4,762672,672B YISHUN AVE 4,15029.982082,993.226458


In [18]:
df_new = df_coordinates.merge(df, on="address", how='outer')
df_new

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70
1,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2015-12,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,275000.0,69
2,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2016-05,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,310000.0,68
3,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2016-06,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,253000.0,68
4,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2016-11,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,290000.0,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155561,NaN,NaN,NaN,NaN,NaN,9 ST. GEORGE'S RD,NaN,NaN,2021-08,KALLANG/WHAMPOA,4 ROOM,9,ST. GEORGE'S RD,04 TO 06,84.0,Simplified,1985,425000.0,62 years 09 months
155562,NaN,NaN,NaN,NaN,NaN,16 ST. GEORGE'S RD,NaN,NaN,2017-11,KALLANG/WHAMPOA,3 ROOM,16,ST. GEORGE'S RD,04 TO 06,67.0,New Generation,1984,323000.0,65 years 10 months
155563,NaN,NaN,NaN,NaN,NaN,16 ST. GEORGE'S RD,NaN,NaN,2020-09,KALLANG/WHAMPOA,4 ROOM,16,ST. GEORGE'S RD,04 TO 06,94.0,New Generation,1984,445000.0,63 years 01 month
155564,NaN,NaN,NaN,NaN,NaN,16 ST. GEORGE'S RD,NaN,NaN,2020-11,KALLANG/WHAMPOA,4 ROOM,16,ST. GEORGE'S RD,04 TO 06,93.0,New Generation,1984,455000.0,62 years 11 months


In [19]:
df_new['resale_price'] = df_new['resale_price'].astype('float')
df_new['floor_area_sqm'] = df_new['floor_area_sqm'].astype('float')
df_new['lease_commence_date'] = df_new['lease_commence_date'].astype('int64')
df_new['lease_remain_years'] = 99 - (2021 - df_new['lease_commence_date'])

df_new.dropna(inplace=True)

df_new

,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,lease_remain_years
0,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70,64
1,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2015-12,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,10 TO 12,60.0,Improved,1986,275000.0,69,64
2,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2016-05,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,69.0,Improved,1986,310000.0,68,64
3,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2016-06,ANG MO KIO,2 ROOM,174,ANG MO KIO AVE 4,07 TO 09,45.0,Improved,1986,253000.0,68,64
4,1.37509746867904,103.83761896123,174,ANG MO KIO AVENUE 4,560174,174 ANG MO KIO AVE 4,10296.855747,418.541691,2016-11,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,290000.0,68,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155229,1.40268497999974,103.893608971394,226B,SUMANG LANE,822226,226B SUMANG LANE,14046.815910,1233.055078,2022-01,PUNGGOL,5 ROOM,226B,SUMANG LANE,07 TO 09,114.0,Premium Apartment,2018,648000.0,95 years 02 months,96
155230,1.33541493957748,103.865168758339,125,POTONG PASIR AVENUE 1,350125,125 POTONG PASIR AVE 1,5997.772735,645.899682,2022-01,TOA PAYOH,EXECUTIVE,125,POTONG PASIR AVE 1,01 TO 03,161.0,Maisonette,1984,985000.0,61 years 03 months,62
155231,1.41859080728129,103.841823925238,672B,YISHUN AVENUE 4,762672,672B YISHUN AVE 4,15029.982082,993.226458,2022-01,YISHUN,3 ROOM,672B,YISHUN AVE 4,04 TO 06,68.0,Model A,2018,391000.0,95 years 01 month,96
155232,1.41859080728129,103.841823925238,672B,YISHUN AVENUE 4,762672,672B YISHUN AVE 4,15029.982082,993.226458,2022-01,YISHUN,4 ROOM,672B,YISHUN AVE 4,10 TO 12,93.0,Model A,2018,550000.0,95 years 01 month,96


In [20]:
df_new.to_csv('hdb_1990_2022.csv', index = False)